In [2]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import json
import requests 

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [3]:
# Instatiate the ES Client
es_client = Elasticsearch('http://localhost:9200')

In [4]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

### Remote File Ingestion

In [8]:
# Download FAQ Documents

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Local File Ingestion

In [ ]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)

# documents = []

# for course_dict in docs_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)

# for doc in tqdm(documents):
#     es_client.index(index=index_name, document=doc)

### LLM FAQ Ingestion

In [10]:
llm_index_name = "llm-course-questions"
es_client.indices.create(index=llm_index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-course-questions'})

In [11]:
with open('documents-llm.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
for doc in tqdm(documents):
    es_client.index(index=llm_index_name, document=doc)

  0%|          | 0/86 [00:00<?, ?it/s]